<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Gen AI Experiments](https://img.shields.io/badge/Gen%20AI%20Experiments-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://github.com/buildfastwithai/gen-ai-experiments)
[![Gen AI Experiments GitHub](https://img.shields.io/github/stars/buildfastwithai/gen-ai-experiments?style=for-the-badge&logo=github&color=gold)](http://github.com/buildfastwithai/gen-ai-experiments)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1GipQtieTAcrcee--JHsnEl5hqs0jx6ge?usp=sharing)

## Master Generative AI in 8 Weeks
**What You'll Learn:**
- Master cutting-edge AI tools & frameworks
- 6 weeks of hands-on, project-based learning
- Weekly live mentorship sessions
- No coding experience required
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)




# Day 4 — AI AGENT SERIES

On **Day 3**, we explored how basic agents are initialized and understood the crucial role of **LLMs** and **tools** in the Agentic AI cycle.  

Now on **Day 4**, we will take things more to security and agent cycle management.

This notebook showcases a secure, memory-augmented AI agent powered by an LLM. The agent is designed with three core capabilities:

- Encrypted Conversational Memory

- Web Search Tool

- Multi-Turn Conversations

The goal of Day 4 is to build and run this secure, tool-enabled conversational agent, verify that memories are being encrypted at rest, and demonstrate both memory recall and tool usage in action.

## Setup and Installation of required libraries
We'll start by installing the prerequisite libraries that we'll be using in this example.

In [ ]:
!pip install -qU agno cryptography duckduckgo-search rich sqlalchemy ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 99.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 46.0.1 which is incompatible.
bigframes 2.19.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.1 which is incompatible.


## Environment setup

In [ ]:
from google.colab import userdata
import os
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
os.environ["MEMORY_AES256_KEY_B64"] = "ZshdX+bSlYY9XvE1gf+g3m5XX4P6JltaiVX9R/fA02g=" # or set any base64 encoding key

### Import necessary libraries

In [ ]:
# --- packages ---
import os, json, base64
from typing import Any, Dict, List
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.db.sqlite import SqliteDb
from rich.pretty import pprint
from agno.tools.duckduckgo import DuckDuckGoTools



## 🔒 Implementing an Encrypted Memory Database Wrapper

In this section, we define a custom EncryptedDb class that wraps around Agno’s SqliteDb (or PostgresDb) to ensure all sensitive fields (memory, content, value, data) are stored encrypted using AES-GCM.

This guarantees that while the agent can still read/write memories normally, the database file only contains ciphertext for personal information.


In [ ]:
# The helper functions _b64 and _b64d handle Base64 encoding/decoding for safe storage of binary data inside the database.
def _b64(x: bytes) -> str:
    return base64.b64encode(x).decode("utf-8")

def _b64d(s: str) -> bytes:
    return base64.b64decode(s.encode("utf-8"))

class EncryptedDb:
    """
    Decorator around Agno's DB (SqliteDb/PostgresDb) that encrypts/decrypts
    selected fields using AES-GCM, while preserving Agno's expected types.
    """
    def __init__(self, inner, key: bytes, fields=("memory","content","value","data")):
        assert len(key) in (16,24,32), "AES key must be 128/192/256 bits"
        self.inner = inner
        self.aesgcm = AESGCM(key)
        self.fields = set(fields)

    # -- low-level helpers --

    # _enc_blob encrypts any field value into a JSON blob containing ciphertext and nonce.
    def _enc_blob(self, plaintext: Any) -> str:
        pt = json.dumps(plaintext).encode("utf-8")
        nonce = os.urandom(12)
        ct = self.aesgcm.encrypt(nonce, pt, None)
        return json.dumps({"__enc__":"aesgcm","n":_b64(nonce),"c":_b64(ct),"v":1})

    # _dec_blob reverses the process, decrypting stored values back into usable plaintext.
    def _dec_blob(self, maybe_cipher: Any) -> Any:
        blob = None
        if isinstance(maybe_cipher, str):
            try:
                blob = json.loads(maybe_cipher)
            except Exception:
                return maybe_cipher
        elif isinstance(maybe_cipher, dict):
            blob = maybe_cipher
        else:
            return maybe_cipher
        if not (isinstance(blob, dict) and blob.get("__enc__") == "aesgcm"):
            return maybe_cipher
        nonce = _b64d(blob["n"]); ct = _b64d(blob["c"])
        pt = self.aesgcm.decrypt(nonce, ct, None)
        return json.loads(pt.decode("utf-8"))

    def _encrypt_obj_fields(self, obj: Dict[str,Any]) -> Dict[str,Any]:
        out = dict(obj)
        for k in list(out.keys()):
            if k in self.fields and out[k] is not None:
                out[k] = self._enc_blob(out[k])
        return out

    def _decrypt_obj_fields(self, obj: Dict[str,Any]) -> Dict[str,Any]:
        out = dict(obj)
        for k,v in list(out.items()):
            if k in self.fields and v is not None:
                out[k] = self._dec_blob(v)
        return out

    def _decrypt_record(self, row):
        # Keep Pydantic types intact
        if hasattr(row, "model_dump") and hasattr(row, "model_copy"):
            data = row.model_dump()
            data = self._decrypt_obj_fields(data)
            return row.model_copy(update=data)
        elif isinstance(row, dict):
            return self._decrypt_obj_fields(row)
        return row

    # -- delegated methods --
    def clear_memories(self, *args, **kwargs):
        return self.inner.clear_memories(*args, **kwargs)

    def get_user_memories(self, user_id: str, *args, **kwargs):
        rows = self.inner.get_user_memories(user_id=user_id, *args, **kwargs)
        return [self._decrypt_record(r) for r in rows]

    def add_user_memory(self, user_id: str, memory: dict, *args, **kwargs):
        enc = self._encrypt_obj_fields(memory)
        return self.inner.add_user_memory(user_id=user_id, memory=enc, *args, **kwargs)

    def save_memory(self, *args, **kwargs):
        if "memory" in kwargs:
            kwargs["memory"] = self._encrypt_obj_fields(kwargs["memory"])
        return getattr(self.inner, "save_memory")(*args, **kwargs)

    def upsert_memory(self, *args, **kwargs):
        if "memory" in kwargs:
            kwargs["memory"] = self._encrypt_obj_fields(kwargs["memory"])
        return getattr(self.inner, "upsert_memory")(*args, **kwargs)

    def __getattr__(self, name):
        return getattr(self.inner, name)


BASE_DB = SqliteDb(db_file="agent_memories.db", memory_table="user_memories")
# AES-256 key (auto-generate for demo if not provided)
KEY_B64 = os.getenv("MEMORY_AES256_KEY_B64")
if not KEY_B64:
    # Demo-only fallback; for prod use env/secret manager
    KEY_B64 = base64.b64encode(os.urandom(32)).decode()
    print("[demo] Generated ephemeral AES-256 key.")
KEY = base64.b64decode(KEY_B64)

DB = EncryptedDb(BASE_DB, key=KEY, fields=("memory","content","value","data"))

###Create Agent

In [ ]:
memory_agent = Agent(
    model=OpenAIChat(id="gpt-4.1"),
    db=DB,
    tools=[DuckDuckGoTools(all=True)],  # <-- enable web search
    enable_agentic_memory=True,
    enable_user_memories=True,
    # Tell the agent to (a) use tools, (b) store personal facts
    description=(
        "You are a helpful assistant with memory and tools. "
        "When a query needs live info, call the web_search tool. "
        "When the user shares stable personal facts (name, location, preferences), "
        "store them as memories and use them in later answers."
    ),
    markdown=True,
)

###Get Started

In [ ]:
# =========================
# Demo flow: shows memory + tool usage
# =========================
user_id = "ava"

# Start clean
DB.clear_memories()

print("\n--- 1) MEMORY WRITE DEMO ---")
memory_agent.print_response(
    "My name is Ava and I like to ski.",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("\n[Decrypted memories now in DB]")
pprint(memory_agent.get_user_memories(user_id=user_id))

print("\n--- 2) TOOL USE DEMO (forces web_search) ---")
memory_agent.print_response(
    "Use the web_search tool to find 3 recent articles about beginner ski tips. "
    "Return titles and links only.",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,  # you'll see a Tool Call block
)

print("\n--- 3) MEMORY RECALL DEMO ---")
memory_agent.print_response(
    "What is my name and what sport do I like?",
    user_id=user_id,
    stream=True,
)